In [5]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import types as t

In [6]:
# Helper functions
def stripText(text):
    x =[]
    return x.append(text)
# Convert function to UDF
stripTextUDF = f.udf(lambda z: stripText(z))

In [8]:
spark = SparkSession.builder \
    .appName("Redit Summarization")\
    .master("yarn")\
    .config("spark.executor.memoryOverhead","2048")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.2.3,org.apache.hadoop:hadoop-aws:2.7.3")\
    .getOrCreate()

Exception in thread "main" org.apache.spark.SparkException: Master must either be yarn or start with spark, mesos, k8s, or local
	at org.apache.spark.deploy.SparkSubmit.error(SparkSubmit.scala:972)
	at org.apache.spark.deploy.SparkSubmit.prepareSubmitEnvironment(SparkSubmit.scala:238)
	at org.apache.spark.deploy.SparkSubmit.org$apache$spark$deploy$SparkSubmit$$runMain(SparkSubmit.scala:898)
	at org.apache.spark.deploy.SparkSubmit.doRunMain$1(SparkSubmit.scala:180)
	at org.apache.spark.deploy.SparkSubmit.submit(SparkSubmit.scala:203)
	at org.apache.spark.deploy.SparkSubmit.doSubmit(SparkSubmit.scala:90)
	at org.apache.spark.deploy.SparkSubmit$$anon$2.doSubmit(SparkSubmit.scala:1043)
	at org.apache.spark.deploy.SparkSubmit$.main(SparkSubmit.scala:1052)
	at org.apache.spark.deploy.SparkSubmit.main(SparkSubmit.scala)


RuntimeError: Java gateway process exited before sending its port number

In [ ]:
# Read data
df = spark.read.format("csv").option("header","true").load("hdfs://namenode:9000/cleaned_data.csv")

In [ ]:
df_top_comments_ids = df.filter(df.link_id == df.parent_id).rdd.map(lambda x: x.link_id).collect()

In [ ]:
df.columns

In [ ]:
df_grouped = df.groupBy("created_utc","subreddit","parent_id").agg(f.sum("ups").alias("total_ups"),f.concat_ws(", ", f.collect_list(df.clean_body)).alias("comment"))

In [ ]:
df_grouped.show(10, truncate = True)

In [ ]:
df_grouped_max = df_grouped.groupBy("created_utc","subreddit").agg(f.max("total_ups").alias("max_ups"), f.first("parent_id").alias("parent_id"),f.first("comment").alias("comment"))

In [ ]:
df_grouped_max.filter(df_grouped_max.subreddit=="AskReddit").show(10, truncate=False)

In [ ]:
df_grouped_max.write.option("header","true").csv("hdfs://namenode:9000/popular_comments.csv")